# Démo pour "jouer" avec les algo de crypto et les certificats utisés par les sites web.


In [60]:
import socket
import struct
from cryptography.hazmat.primitives.asymmetric import rsa, padding
from cryptography.hazmat.primitives.ciphers.aead import AESGCM
from cryptography.hazmat.primitives import hashes
from cryptography.x509 import load_der_x509_certificate
import ssl

In [61]:
HOST = "www.python.org"
PORT = 443

## Étape 1 : Connexion TCP au serveur

In [62]:
sock = socket.create_connection((HOST, PORT))

## Etape 2: Forger et envoyer le client Hello
Le pack |I désigne le byte order ici (Reseau)(Unsigned Int)

In [63]:
cipher_suites = [b"TLS_RSA_WITH_AES_256_GCM_SHA256", b"TLS_RSA_WITH_AES_128_GCM_SHA256"]
client_hello = b"TLS_CLIENT_HELLO" + b"".join(cipher_suites)
sock.sendall(struct.pack("!I", len(client_hello)) + client_hello)

## Etape 3: Réception du serveur Hello

In [64]:
server_hello_size = struct.unpack("!I", sock.recv(4))[0]
server_hello = sock.recv(server_hello_size)
selected_cipher_hex = server_hello.hex()  # Affichage brut en hexadécimal
print("ServerHello brut (hex) :", selected_cipher_hex)

try:
    selected_cipher = server_hello.decode('utf-8').strip()
except UnicodeDecodeError:
    selected_cipher = selected_cipher_hex  # Utilisation de l'hex en cas d'échec
print("ServerHello reçu, suite choisie :", selected_cipher)

cipher_details = {
    "TLS_RSA_WITH_AES_256_GCM_SHA256": "RSA + AES-256-GCM + SHA-256",
    "TLS_RSA_WITH_AES_128_GCM_SHA256": "RSA + AES-128-GCM + SHA-256"
}
print("Détails de la suite de chiffrement :", cipher_details.get(selected_cipher, "Inconnue"))

ServerHello brut (hex) : 02023215030300020100
ServerHello reçu, suite choisie : 2  
Détails de la suite de chiffrement : Inconnue
